In [ ]:
#using calysto bash kernel

In [7]:
conda activate scan-04-phasing

(scan-04-phasing) 


In [8]:
mkdir -p /master/nplatt/sch_hae_scan/results/phasing && cd /master/nplatt/sch_hae_scan/results/phasing

(scan-04-phasing) 


In [ ]:
sleep 5h

In [ ]:
    
#calculate a recomb map.  1cm = 287Kb based on data from Criscione et al. (2009) avg
cat ../mapping_rates/mappable_snvs.vcf | awk '!/^#/ {printf "%s\t%s\t%.6f\t%s\n", $1, $3, $2/287000, $2}' | grep NC > gen.map

In [ ]:
#get major chroms
cut -f1 gen.map | grep NC_067 | sort | uniq >chrs.list

In [ ]:
#make dirs for each contig/chrom
for CHR in $(cat chrs.list); do
    mkdir $CHR
done

In [ ]:
#print chrom specific recomb maps
awk '{print $0 > $1"/"$1".gen.map" }' gen.map

In [ ]:
cp /master/nplatt/sch_hae_scan/data/header.sge.sh .
cat  header.sge.sh

In [ ]:
#impute on each seperatley (create seperate qsub scripts)
for CHR in $(cat chrs.list); do

    cd $CHR

    sed "s/CHR/$CHR/" ../header.sge.sh >$CHR.sge.sh

    #extract autosome specific vcf for the samples of interest
    CMD="vcftools \
        --vcf /master/nplatt/sch_hae_scan/results/mapping_rates/mappable_snvs.vcf \
        --chr $CHR \
        --recode \
        --recode-INFO-all \
        --stdout \
        >$CHR.vcf"
    echo -e $CMD"\n" >>$CHR.sge.sh

    CMD="java -Xmx64g -jar /master/nplatt/anaconda3/envs/scan-04-phasing/share/beagle-5.2_21Apr21.304-0/beagle.jar \
        gt=$CHR.vcf \
        out="$CHR"_beagle \
        map=$CHR.gen.map \
        nthreads=96 \
        window=20 \
        overlap=10 \
        iterations=60 \
        burnin=20 \
        >$CHR.log 2>&1"
    echo -e $CMD"\n" >>$CHR.sge.sh

    CMD="gunzip "$CHR"_beagle.vcf.gz"
    echo -e $CMD"\n" >>$CHR.sge.sh

    qsub $CHR.sge.sh
    cd ..

done

### ... wait for everything to phase

In [9]:
#create one file of phased snps (for all autosomes)
vcfcombine $(ls NC_067*.1/*beagle.vcf) >beagle.vcf

(scan-04-phasing) (scan-04-phasing) 
